In [1]:
from models.cifar_resnet import cifar_resnet
import torch
from pprint import pprint

In [2]:
cresnet8 = cifar_resnet(variant=8).to(torch.device('cuda'))
cresnet8.eval()
print('Initialised resnet for evaluation')

Initialised resnet for evaluation


In [3]:
from fed_df_data_loader.get_crops_dataloader import get_distill_imgloader

distill_dataloader = get_distill_imgloader('./data/single_img_crops/crops', dataset_name='cifar10', batch_size=1024)

In [4]:
from strategy.clustering import cluster_embeddings

In [5]:
device = torch.device('cuda')
test_df,score = cluster_embeddings(dataloader=distill_dataloader, model=cresnet8, device=device, seed=42)

In [6]:
print(score)
test_df.head()

22042.80078125


,img,cluster,cluster_dist
img_no,,,
0,[[[[-2.390296 -2.4290657 -2.4290657 -2.429...,5,0.475970
1,[[[[ 1.4479177 1.5836121 0.8469853 2.126...,7,0.609089
2,[[[[-2.4290657 -2.4096808 1.4673026 1.622...,7,0.351351
3,[[[[-0.8395026 -0.95581204 0.24605286 1.215...,7,0.559803
4,[[[[ 1.5836121 1.370378 0.90514 0.827...,0,0.730672


In [7]:
from strategy.clustering import prune_clusters

In [8]:
test_df2 = prune_clusters(test_df,n_crops=3000,heuristic_percentage="60-40")
test_df2.head()


,img,cluster,cluster_dist
img_no,,,
0,[[[[ 1.3316082 0.98267967 0.5368266 0.924...,9,0.395222
1,[[[[-2.351526 -1.9444429 -1.6342843 0.246...,0,0.417092
2,[[[[ 2.4559333 2.4559333 2.514088 2.514...,3,0.306361
3,[[[[ 2.514088 2.514088 2.3202388 1.971...,9,0.351085
4,[[[[ 1.8937707 1.350993 0.98267967 0.691...,7,0.213088


In [9]:
from strategy.clustering import visualise_clusters

In [10]:
with open('test_img.png','wb') as f:
    visualise_clusters(test_df2,f,n_imgs=8)

In [11]:
from strategy.clustering import prepare_for_transport

out = prepare_for_transport(test_df2)
print(type(out))

<class 'bytes'>


In [12]:
from strategy.clustering import extract_from_transport

img_loader = extract_from_transport(out)
imgs, labels = next(iter(img_loader))
print(imgs.size())
print(imgs[0].size())
print(labels[0])

torch.Size([512, 3, 32, 32])
torch.Size([3, 32, 32])
tensor(1)
